## Task 1: News Topic Classifier Using BERT
### Objective:
Fine-tune a transformer model (e.g., BERT) to classify news headlines into topic categories.

### Dataset: AG News Dataset (Available on Hugging Face Datasets)


### Instructions:
    ● Tokenize and preprocess the dataset
    ● Fine-tune the bert-base-uncased model using Hugging Face Transformers
    ● Evaluate the model using accuracy and F1-score
    ● Deploy the model using Streamlit or Gradio for live interaction
    Skills Gained:
    ● NLP using Transformers
    ● Transfer learning & fine-tuning
    ● Evaluation metrics for text classification
    ● Lightweight model deployment


## Loading dataset

In [1]:
# ③ Known‑good combo
# !pip install --force-reinstall "datasets==2.16.0" "fsspec==2023.9.2"


In [2]:
from datasets import load_dataset

# Now re-load the dataset
dataset = load_dataset("ag_news")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [3]:
dataset['train']  # text + description hase been concatenated into text column

Dataset({
    features: ['text', 'label'],
    num_rows: 120000
})

In [4]:
dataset['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 7600
})

##  Tokenization

In [5]:
from transformers import BertTokenizer

tokenizer= BertTokenizer.from_pretrained('bert-base-uncased')

def tokenizer_function(data_text):
  return tokenizer(data_text["text"], padding='max_length', truncation=True)


tokenized_dataset =dataset.map(tokenizer_function , batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

## Prepare Data for PyTorch

In [6]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Padding each sentence in a batch to match the longest sentence in that batch.

# Automatically creates attention masks.

# Returns a PyTorch dictionary: {input_ids, attention_mask, labels}

## Load Pretrained Model for Classification

In [7]:
from transformers import BertForSequenceClassification

model=BertForSequenceClassification.from_pretrained('bert-base-uncased' , num_labels=4)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Training setup

In [8]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score

#metric computation

def compute_metrics(eval_pred):
  logits, labels= eval_pred   # logits= raw output before softmax
  predictions=logits.argsmax(axis=-1)

  return {
      "accuracy_score " :accuracy_score(labels, predictions), # actual vs predicted lables
      "F1 Score" : f1_score(labels,predictions, average='weighted')
      }


In [ ]:
training_args= TrainingArguments(
    output_dir='./results',
    run_name="bert-agnews-finetune",
    eval_strategy='epoch',
    save_strategy="epoch",             # Save model at the end of each epoch
    save_total_limit=2,                   ## Only keep last 2 checkpoints to save disk
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
)

trainer= Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: wasidkhan123007 (wasidkhan123007-university-of-peshawar) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
eval_results=trainer.evaluate()
print(eval_results)

In [ ]:
trainer.save_model("my_finetuned_bert")  # Saves tokenizer and model
tokenizer.save_pretrained("my_finetuned_bert")
